make a corrected version of the code addressing the "Description, File, Line, Column" formatted toolbox error statements; it is saying: {}

//+------------------------------------------------------------------+
//|                                                      H1_RSI_EA.mq5 |
//|                        Corrected MQL5 Version                     |
//+------------------------------------------------------------------+
#property strict

input double RiskPercent = 1.0;        // Risk per trade in %
input int RSI_Period = 14;             // RSI period
input double RSI_BuyTP = 70;           // RSI overbought for Buy TP
input double RSI_SellTP = 30;          // RSI oversold for Sell TP

// Global variables
datetime lastTradeTime = 0;

//+------------------------------------------------------------------+
//| Expert initialization function                                   |
//+------------------------------------------------------------------+
int OnInit()
{
   return(INIT_SUCCEEDED);
}

//+------------------------------------------------------------------+
//| Expert tick function                                             |
//+------------------------------------------------------------------+
void OnTick()
{
   MqlRates rates[];
   int copied = CopyRates(_Symbol, PERIOD_H1, 0, 24, rates);
   if(copied <= 0) return;

   // Get the time of the latest H1 candle
   datetime latestH1Time = rates[0].time;

   // Only run once per H1 12:00 candle
   int hourOfCandle = TimeHour(latestH1Time);
   if(hourOfCandle == 12 && lastTradeTime != latestH1Time)
   {
      lastTradeTime = latestH1Time;

      double rangeHigh = rates[0].high; // Latest H1 candle high
      double rangeLow  = rates[0].low;  // Latest H1 candle low
      double rangeMid  = (rangeHigh + rangeLow) / 2.0;

      // Calculate lot size for 1% risk
      double stopLossDistance = rangeHigh - rangeMid;
      if(stopLossDistance <= 0) return; // safety check
      double lotSize = CalculateLotSize(stopLossDistance);

      // Place BuyStop
      PlacePendingOrder(ORDER_TYPE_BUY_STOP, rangeHigh, rangeMid, lotSize);

      // Place SellStop
      PlacePendingOrder(ORDER_TYPE_SELL_STOP, rangeLow, rangeMid, lotSize);
   }

   // Manage TP based on RSI
   ManagePositions();
}

//+------------------------------------------------------------------+
//| Calculate lot size based on 1% risk                               |
//+------------------------------------------------------------------+
double CalculateLotSize(double stopDistance)
{
   double riskAmount = AccountInfoDouble(ACCOUNT_EQUITY) * RiskPercent / 100.0;
   double tickValue  = SymbolInfoDouble(_Symbol, SYMBOL_TRADE_TICK_VALUE);
   double tickSize   = SymbolInfoDouble(_Symbol, SYMBOL_TRADE_TICK_SIZE);

   double lotSize = riskAmount / (stopDistance / tickSize * tickValue);

   double minLot = SymbolInfoDouble(_Symbol, SYMBOL_VOLUME_MIN);
   double maxLot = SymbolInfoDouble(_Symbol, SYMBOL_VOLUME_MAX);
   double lotStep = SymbolInfoDouble(_Symbol, SYMBOL_VOLUME_STEP);

   lotSize = MathMax(minLot, MathMin(maxLot, lotSize));
   lotSize = MathFloor(lotSize / lotStep) * lotStep;

   return(lotSize);
}

//+------------------------------------------------------------------+
//| Place pending order                                               |
//+------------------------------------------------------------------+
void PlacePendingOrder(ENUM_ORDER_TYPE type, double price, double sl, double lot)
{
   MqlTradeRequest request;
   MqlTradeResult result;
   ZeroMemory(request);
   ZeroMemory(result);

   request.action   = TRADE_ACTION_PENDING;
   request.symbol   = _Symbol;
   request.volume   = lot;
   request.type     = type;
   request.price    = price;
   request.sl       = sl;
   request.tp       = 0;  // TP handled by RSI logic
   request.deviation = 10;
   request.magic    = 123456;

   if(!OrderSend(request, result))
   {
      Print("Error placing pending order: ", result.retcode);
   }
}

//+------------------------------------------------------------------+
//| Manage open positions with RSI-based TP                          |
//+------------------------------------------------------------------+
void ManagePositions()
{
   static int rsiHandle = iRSI(_Symbol, PERIOD_CURRENT, RSI_Period, PRICE_CLOSE);
   if(rsiHandle == INVALID_HANDLE) return;

   double rsiArray[1];
   if(CopyBuffer(rsiHandle, 0, 0, 1, rsiArray) <= 0) return;
   double rsi = rsiArray[0];

   for(int i = PositionsTotal() - 1; i >= 0; i--)
   {
      ulong ticket = PositionGetTicket(i);
      if(!PositionSelectByTicket(ticket)) continue;

      double tpPrice = 0;

      if(PositionGetInteger(POSITION_TYPE) == POSITION_TYPE_BUY && rsi >= RSI_BuyTP)
      {
         tpPrice = SymbolInfoDouble(_Symbol, SYMBOL_BID);
      }
      else if(PositionGetInteger(POSITION_TYPE) == POSITION_TYPE_SELL && rsi <= RSI_SellTP)
      {
         tpPrice = SymbolInfoDouble(_Symbol, SYMBOL_ASK);
      }

      if(tpPrice != 0)
      {
         MqlTradeRequest modify;
         MqlTradeResult result;
         ZeroMemory(modify);
         ZeroMemory(result);

         modify.action   = TRADE_ACTION_SLTP;
         modify.position = ticket;
         modify.tp       = tpPrice;

         if(!OrderSend(modify, result))
         {
            Print("Error modifying position TP: ", result.retcode);
         }
      }
   }
}
